In [3]:
import math
import pickle as p
import tensorflow as tf
import numpy as np
import utils
import json

In [4]:
!conda info -e


# conda environments:
#
base                     /Applications/anaconda3
insight                  /Applications/anaconda3/envs/insight
intro_dl                 /Applications/anaconda3/envs/intro_dl
tf                    *  /Applications/anaconda3/envs/tf
tf2                      /Applications/anaconda3/envs/tf2



In [5]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = tf.add(a, b)
d = tf.subtract(b, 1)
e = tf.multiply(c, d)

In [7]:
with tf.Session() as session:
    a_data, b_data = 3.0, 6.0
    feed_dict = {a: a_data, b: b_data}
    output = session.run([e], feed_dict=feed_dict)
    print(output) # 45.0  

[45.0]


In [10]:
n_input_nodes = 2
n_output_nodes = 1
x = tf.placeholder(tf.float32, (None, n_input_nodes))
W = tf.Variable(tf.ones((n_input_nodes, n_output_nodes)), dtype=tf.float32)
b = tf.Variable(tf.zeros(n_output_nodes), dtype=tf.float32)

'''TODO: Define the operation for z (use tf.matmul to multiply W and x).'''
z = tf.add(tf.matmul(W,x), b)

'''TODO: Define the operation for out (use tf.sigmoid).'''
out = tf.sigmoid(z)

In [11]:
test_input = [[0.5, 0.5]]
with tf.Session() as session:
    tf.global_variables_initializer().run(session=session)
    feed_dict = {x: test_input}
    output = session.run([out], feed_dict=feed_dict)
    print(output[0]) # This should output 0.73105. If not, double-check your code above

[[0.62245935 0.62245935]
 [0.62245935 0.62245935]]


In [17]:
# set variables
tweet_size = 20
hidden_units = 256
vocab_size = 8000
batch_size = 32

# this just makes sure that all our following operations will be placed in the right graph.
tf.reset_default_graph()

# create a session variable that we can run later.
session = tf.Session()

In [15]:
# the placeholder for tweets has first dimension batch_size for each tweet in a batch,
# second dimension tweet_size for each word in the tweet, and third dimension vocab_size
# since each word itself is represented by a one-hot vector of size vocab_size.
# Note that we use 'None' instead of batch_size for the first dimsension.  This allows us 
# to deal with variable batch sizes
tweets = tf.placeholder(tf.float32, [None, tweet_size, vocab_size])

'''TODO: create a placeholder for the labels (our predictions).  
   This should be a 1D vector with size = None, 
   since we are predicting one value for each tweet in the batch,
   but we want to be able to deal with variable batch sizes.''';
labels = tf.placeholder(tf.float32,[None])

In [26]:
'''TODO: create 2 LSTM Cells using BasicLSTMCell.  Note that this creates a *layer* of LSTM
   cells, not just a single one.''';
lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(hidden_units)
lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(hidden_units)


'''TODO: create three LSTM layers by wrapping two instances of 
   lstm_cell from above in tf.contrib.rnn_cell.MultiRNNCell. Note that
   you can use multiple cells by doing [cell1, cell2]. Also note
   that you should use state_is_tuple=True as an argument.  This will allow
   us to access the part of the cell state that we need later on.''';
multi_lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1,lstm_cell_2], state_is_tuple = True)


'''TODO: define the operation to create the RNN graph across time.  
   tf.nn.dynamic_rnn dynamically constructs the graph when it is executed,
   and returns the final cell state.''';
_, final_state = tf.nn.dynamic_rnn (multi_lstm_cells, initial_state, dtype=tf.float32)

ValueError: Shape (256, ?) must have rank at least 3

In [ ]:
## We define this function that creates a weight matrix + bias parameter
## and uses them to do a matrix multiplication.
def linear(input_, output_size, name, init_bias=0.0):
    shape = input_.get_shape().as_list()
    with tf.variable_scope(name):
        W = tf.get_variable("weights", [shape[-1], output_size], tf.float32, tf.random_normal_initializer(stddev=1.0 / math.sqrt(shape[-1])))
    if init_bias is None:
        return tf.matmul(input_, W)
    with tf.variable_scope(name):
        b = tf.get_variable("bias", [output_size], initializer=tf.constant_initializer(init_bias))
    return tf.matmul(input_, W) + b

In [ ]:
'''TODO: pass the final state into this linear function to multiply it 
   by the weights and add bias to get our output.
   
   {Quick note that we need to feed in final_state[-1][-1] into linear since 
   final_state is actually a tuple consisting of the cell state 
   (used internally for the cell to keep track of things) 
   as well as the hidden state (the output of the cell), and one of these 
   tuples for each layer. We want the hidden state for the last layer, so we use 
   final_state[-1][-1]}''';

sentiment = linear(final_state[-1][-1], )